In [3]:
import os, sys, inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

from brian2 import *
from matplotlib import pyplot as plt
import Documentation.parameters as p
from Foundations.helpers import scale_input_theory, make_spiketrain

from Foundations.MI import analyze_exp
from Old_testing.make_dynamic_experiment_ori import make_dynamic_experiments
from Old_testing.currentmodel import Barrel_PC, Barrel_IN
import pandas as pd

defaultclock = p.dt*ms

In [4]:
[input_theory, hidden_state] = make_dynamic_experiments(p.qon_qoff_type, p.baseline, p.tau, p.factor_ron_roff, p.mean_firing_rate, p.sampling_rate, p.duration, seed)

TypeError: Cannot cast scalar from dtype('O') to dtype('int64') according to the rule 'safe'

In [ ]:


inj_current = scale_input_theory(input_theory, 0, scale, p.dt)

start_scope()
#Initialising the neuron classes
PC = Barrel_PC(p.dt, inj_current)
IN = Barrel_IN(p.dt, inj_current)
#Getting the neuron groups
PC_neuron = PC.getNeurongroup()
IN_neuron = IN.getNeurongroup()

net2 = Network(PC_neuron,
               IN_neuron)

PC_M = StateMonitor(PC_neuron, 'v', record=True)
PC_S = SpikeMonitor(PC_neuron, 'v', record = True)
IN_M = StateMonitor(IN_neuron, 'v', record=True)
IN_S = SpikeMonitor(PC_neuron, 'v', record = True)

net2.add(PC_M, IN_M, PC_S, IN_S)
#getting all the parameters for in the synapse
delay = 0.6

syn2 = Synapses(IN_neuron, PC_neuron, on_pre='''v_post += -w1*mV''', delay= delay*ms, dt=p.dt * ms)
syn2.connect(i=0, j=1)
net2.add(syn2)

net2.store()

output_PC = pd.DataFrame()
output_IN = pd.DataFrame()
samples = p.duration/p.dt

for w1 in weights:
    net2.restore()
    w1 = w1
    net2.run(p.duration*ms)
    plotting(PC_M, IN_M, w1, scale)
    spiketrain = make_spiketrain(PC_S)
    output_PC = pd.concat([output_PC, analyze_exp(hidden_state, input_theory,0, spiketrain,samples, w1, scale)], axis = 0)
    output_IN = pd.concat([output_IN, analyze_exp(hidden_state, input_theory,0, spiketrain,samples, w1, scale)], axis = 0)

return output_PC, output_IN, spiketrain